In [1]:
import numpy as np

# Item XIX

Implement the Conjugate Gradient Method with symmetric preconditioner. Select a convenient problem of your choice, but not a trivial one and test it. Please describe the problem and implementation completely.

---

In [2]:
# This is a regular version of the Conjugate Gradient, but generalized
# so the way that the multiplications by the matrix A are done can be
# be replaced
def conjugate_gradient(A,b,pinv=lambda x:x,x0=None):
    n = A.shape[0]
    if x0 is None: x0 = np.zeros(n)
    #
    r0 = b - np.dot(A,x0)
    z0 = pinv(r0)
    p0 = z0
    #
    xk = x0
    rk = r0
    pk = p0
    zk = z0
    #
    for k in range(n):
        ak = np.dot(rk,zk)/np.dot(pk,np.dot(A,pk))
        xk1 = xk + ak*pk
        rk1 = rk - ak*np.dot(A,pk)
        zk1 = pinv(rk1)
        bk = np.dot(zk1,rk1)/np.dot(zk,rk)
        pk1 = zk1 + bk*pk
        # Move forward
        xk = xk1
        rk = rk1
        pk = pk1
        zk = zk1
    return xk1

In [3]:
def random_positive_definite_matrix(n):
    a = np.random.random((n,n))
    A = np.dot(a,a.T)
    np.fill_diagonal(A,np.diag(A)+n)
    return A
N = 5
A = random_positive_definite_matrix(N)
b = np.random.random(N)
print(A)
print(b)
x1 = np.linalg.solve(A,b)
x2 = conjugate_gradient(A,b)
print(x1)
print(x2)

[[6.09190748 0.61823363 1.05819101 0.93441106 1.03996343]
 [0.61823363 6.14585374 0.98062608 0.85109716 1.07210015]
 [1.05819101 0.98062608 6.7145223  1.38233099 1.39712506]
 [0.93441106 0.85109716 1.38233099 6.69908011 1.75988505]
 [1.03996343 1.07210015 1.39712506 1.75988505 6.94552285]]
[0.53405442 0.16455434 0.46100054 0.18008056 0.39653179]
[ 0.0728593   0.00572985  0.04956425 -0.0037683   0.03628267]
[ 0.0728593   0.00572985  0.04956425 -0.0037683   0.03628267]


Let's consider the Jacobi preconditioner (the preconditioner is the diagonal of the matrix, so the inverse is simple):

In [4]:
def jacobi_preconditioner_inv(A):
    P = np.diag(A)**-1
    return lambda x: P*x

In [5]:
NS = (10,100,1000)

for nn in NS:
    AA = random_positive_definite_matrix(nn)
    bb = np.random.random(nn)
    xx = conjugate_gradient(AA,bb)
    xx2 = conjugate_gradient(AA,bb,pinv=jacobi_preconditioner_inv(AA))
    # Check maximum error:
    max_err = np.max(np.abs(np.dot(AA,xx)-bb))
    print(max_err)
    max_err2 = np.max(np.abs(np.dot(AA,xx2)-bb))
    print(max_err2)

1.4988010832439613e-14
3.164135620181696e-14
6.661338147750939e-16
7.771561172376096e-16


/home/fcasas/Music/p36cpu/lib/python3.6/site-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars


nan
nan
